# *SOURCE*
https://www.youtube.com/watch?v=doDUihpj6ro&t=7339s
https://github.com/nicknochnack/ActionDetectionforSignLanguage

In [45]:

!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [10]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('/content/Tutorial_LSTM_MP/MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [6]:
! git clone https://github.com/nickbp760/Tutorial_LSTM_MP.git

Cloning into 'Tutorial_LSTM_MP'...
remote: Enumerating objects: 2840, done.
remote: Counting objects: 100% (2840/2840), done.
remote: Compressing objects: 100% (2807/2807), done.
remote: Total 2840 (delta 36), reused 2834 (delta 30), pack-reused 0
Receiving objects: 100% (2840/2840), 20.95 MiB | 16.92 MiB/s, done.
Resolving deltas: 100% (36/36), done.


### **Preprocess Data and Create Labels and Features**

The label map containts the action label

In [7]:
label_map = {label:num for num, label in enumerate(actions)}

In [8]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

load the the data that have been prepared before

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

1662 is an amount of features in one frame, 30 is an amount frames for one video action, 90 is an amount of 
video actions

In [14]:
468*3+33*4+21*3+21*3

1662

In [15]:
np.array(sequences).shape

(90, 30, 1662)

DEFINE X (Input) and y (Output)

In [21]:
X = np.array(sequences)
print(X.shape)
y = to_categorical(labels).astype(int)
print(y.shape)

(90, 30, 1662)
(90, 3)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

### **Build and Train LSTM Neural Network**

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [25]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [26]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [27]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 3s 250ms/step - loss: 3.5819 - categorical_accuracy: 0.3679
Epoch 2/2000
3/3 [==============================] - 0s 108ms/step - loss: 11.2633 - categorical_accuracy: 0.3914
Epoch 3/2000
3/3 [==============================] - 0s 101ms/step - loss: 7.5264 - categorical_accuracy: 0.3385
Epoch 4/2000
3/3 [==============================] - 0s 104ms/step - loss: 5.1518 - categorical_accuracy: 0.3484
Epoch 5/2000
3/3 [==============================] - 0s 103ms/step - loss: 3.9528 - categorical_accuracy: 0.2583
Epoch 6/2000
3/3 [==============================] - 0s 106ms/step - loss: 4.4301 - categorical_accuracy: 0.2114
Epoch 7/2000
3/3 [==============================] - 0s 120ms/step - loss: 5.0168 - categorical_accuracy: 0.3463
Epoch 8/2000
3/3 [==============================] - 0s 110ms/step - loss: 4.6586 - categorical_accuracy: 0.3522
Epoch 9/2000
3/3 [==============================] - 0s 104ms/step - loss: 3.8480 - categorical_accuracy

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

In [47]:
model.save('/content/Tutorial_LSTM_MP/Weight_model/action.h5')

### **Make One Predictions**

In [29]:
res = model.predict(X_test)

In [30]:

actions[np.argmax(res[4])]

'thanks'

In [31]:
actions[np.argmax(y_test[4])]

'thanks'

### **ReLoad Model**

In [48]:
del model

In [49]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [50]:
model.load_weights('/content/Tutorial_LSTM_MP/Weight_model/action.h5')

### **Evaluation using Confusion Matrix and Accuracy**

In [41]:
y_test_predict = model.predict(X_test)

In [42]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(y_test_predict, axis=1).tolist()

In [43]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1, 1],
        [0, 3]],

       [[3, 0],
        [1, 1]]])

In [44]:
accuracy_score(ytrue, yhat)

0.8